<a href="https://colab.research.google.com/github/Sato-Eishiro/-_-_/blob/main/%E3%82%B7%E3%83%95%E3%83%88%E6%9C%80%E9%81%A9%E5%8C%96%E5%95%8F%E9%A1%8C_%E4%BD%90%E8%97%A4%E8%A9%A0%E5%8F%B2%E9%83%8E_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 23.9 MB/s eta 0:00:00


In [5]:
from pulp import LpProblem, LpVariable, lpSum, LpMinimize, LpBinary
import pandas as pd

# 従業員リスト
employees = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J"]

# 28日間（1日目が月曜日）
days = list(range(1, 29))
shifts = ["朝", "夜"]

# 各従業員の出勤可能回数（週換算）
employee_weekly_limits = {
    "A": 5, "B": 4, "C": 2, "D": 2, "E": 3,
    "F": 4, "G": 4, "H": 4, "I": 4, "J": 2
}

# 月あたりの最大勤務回数（Fさんのみ）
max_shifts_f_per_month = 10

# 朝夜両方OKな従業員
double_shift_allowed = {"A", "B", "E", "F", "H", "I", "J"}

# 数理最適化モデル
prob = LpProblem("Shift_Scheduling", LpMinimize)

# 変数: x[従業員, 日, シフト]（1なら割り当て、0ならなし）
x = {(i, j, t): LpVariable(f"x_{i}_{j}_{t}", cat=LpBinary)
     for i in employees for j in days for t in shifts}

# 補助変数: 各従業員の希望勤務日数との差の絶対値を表現
y = {i: LpVariable(f"y_{i}", lowBound=0) for i in employees}

# 目的関数: 各従業員の希望日数と実際の日数の差を最小化
prob += lpSum(y[i] for i in employees)

# 制約条件
for j in days:
    weekday = (j - 1) % 7  # 0:月, 1:火, ..., 5:土, 6:日
    for t in shifts:
        if weekday in range(5):  # 平日（月～金）
            prob += lpSum(x[i, j, t] for i in employees) == 2
        else:  # 土日
            prob += lpSum(x[i, j, t] for i in employees) == 3

for i in employees:
    # 各従業員の週換算での勤務回数制限
    max_shifts_per_month = employee_weekly_limits[i] * 4
    prob += lpSum(x[i, j, t] for j in days for t in shifts) <= max_shifts_per_month

    # 朝と夜両方に入れるのはdouble_shift_allowed のみ
    if i not in double_shift_allowed:
        for j in days:
            prob += lpSum(x[i, j, t] for t in shifts) <= 1  # 朝夜両方のシフトに入れない

    # Fさんは月10回まで
    if i == "F":
        prob += lpSum(x[i, j, t] for j in days for t in shifts) <= max_shifts_f_per_month

    # 希望勤務日数との差の絶対値を補助変数yで表現
    prob += lpSum(x[i, j, t] for j in days for t in shifts) - max_shifts_per_month <= y[i]
    prob += max_shifts_per_month - lpSum(x[i, j, t] for j in days for t in shifts) <= y[i]

    # 新人（C, D, G）は朝のみ不可
    if i in ["C", "D", "G"]:
        for j in days:
            prob += lpSum(x[i, j, "朝"] for t in shifts) == 0

# 最適化実行
prob.solve()

# 実際の勤務回数を計算
actual_shifts = {i: 0 for i in employees}
for i in employees:
    for j in days:
        for t in shifts:
            if x[i, j, t].value() == 1:
                actual_shifts[i] += 1

# 希望勤務回数（expected_shifts）は週換算の値から計算
expected_shifts = {i: employee_weekly_limits[i] * 4 for i in employees}

# 差分を計算
deviation = {i: abs(actual_shifts[i] - expected_shifts[i]) for i in employees}

# 結果を表示
schedule_dict = {f'{i}日目': {'朝': [], '夜': []} for i in range(1, 29)}

# 出力結果を格納
for i in employees:
    for j in days:
        for t in shifts:
            if x[i, j, t].value() == 1:
                schedule_dict[f'{j}日目'][t].append(i)

# pandas DataFrameを作成
df_schedule = pd.DataFrame(schedule_dict)

# 表示
print(df_schedule)

      1日目     2日目     3日目     4日目     5日目        6日目        7日目     8日目  \
朝  [A, F]  [A, H]  [B, H]  [F, I]  [H, J]  [B, H, I]  [A, E, I]  [A, B]   
夜  [A, G]  [F, I]  [A, D]  [A, E]  [D, G]  [E, G, J]  [A, D, G]  [B, G]   

      9日目    10日目  ...    19日目       20日目       21日目    22日目    23日目    24日目  \
朝  [A, F]  [H, I]  ...  [B, F]  [B, E, J]  [A, I, J]  [B, F]  [H, I]  [B, E]   
夜  [A, G]  [B, F]  ...  [C, I]  [C, G, I]  [C, D, I]  [C, D]  [B, H]  [E, G]   

     25日目    26日目       27日目       28日目  
朝  [H, I]  [E, J]  [E, I, J]  [A, E, I]  
夜  [B, G]  [A, D]  [A, B, G]  [C, G, H]  

[2 rows x 28 columns]


In [6]:
# 評価結果の出力
print("=== 各従業員の勤務回数 ===")
for i in employees:
    print(f"{i}: 実際 {actual_shifts[i]} 回 / 希望 {expected_shifts[i]} 回 (差: {deviation[i]})")

# Fさんの勤務回数チェック
if actual_shifts["F"] > max_shifts_f_per_month:
    print(f"⚠️ Fさんの勤務回数が制限 ({max_shifts_f_per_month}回) を超えています！")
else:
    print(f"✅ Fさんの勤務回数: {actual_shifts['F']}回（制限内）")

# シフトの偏りチェック（朝と夜のバランス）
shift_distribution = {"朝": 0, "夜": 0}
for j in days:
    for t in shifts:
        shift_distribution[t] += sum(x[i, j, t].value() for i in employees)

print("\n=== シフトの総数 ===")
print(f"朝のシフト: {shift_distribution['朝']} 回")
print(f"夜のシフト: {shift_distribution['夜']} 回")

c_morning_shifts = sum(x["C", j, "朝"].value() for j in days)
d_morning_shifts = sum(x["D", j, "朝"].value() for j in days)
g_morning_shifts = sum(x["G", j, "朝"].value() for j in days)

if c_morning_shifts == 0:
    print("✅ Cさんは朝シフトに入っていません（希望通り）")
else:
    print(f"⚠️ Cさんが朝に {c_morning_shifts} 回シフトに入っています！（希望と違う）")

if d_morning_shifts == 0:
    print("✅ Dさんは朝シフトに入っていません（希望通り）")
else:
    print(f"⚠️ Dさんが朝に {d_morning_shifts} 回シフトに入っています！（希望と違う）")

if g_morning_shifts == 0:
    print("✅ Gさんは朝シフトに入っていません（希望通り）")
else:
    print(f"⚠️ Gさんが朝に {g_morning_shifts} 回シフトに入っています！（希望と違う）")

=== 各従業員の勤務回数 ===
A: 実際 20 回 / 希望 20 回 (差: 0)
B: 実際 16 回 / 希望 16 回 (差: 0)
C: 実際 6 回 / 希望 8 回 (差: 2)
D: 実際 8 回 / 希望 8 回 (差: 0)
E: 実際 12 回 / 希望 12 回 (差: 0)
F: 実際 10 回 / 希望 16 回 (差: 6)
G: 実際 16 回 / 希望 16 回 (差: 0)
H: 実際 16 回 / 希望 16 回 (差: 0)
I: 実際 16 回 / 希望 16 回 (差: 0)
J: 実際 8 回 / 希望 8 回 (差: 0)
✅ Fさんの勤務回数: 10回（制限内）

=== シフトの総数 ===
朝のシフト: 64.0 回
夜のシフト: 64.0 回
✅ Cさんは朝シフトに入っていません（希望通り）
✅ Dさんは朝シフトに入っていません（希望通り）
✅ Gさんは朝シフトに入っていません（希望通り）
